In [46]:
# coding: utf-8
"""
gensimを用いたLDA
"""

from gensim import corpora, models, similarities
import csv
import MeCab

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()
    
def parsing(sentence):
    mecab = MeCab.Tagger("-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")
    res = mecab.parseToNode(sentence)
    list_words = []
    while res:
        features = res.feature.split(",")
        if (features[0] == "名詞" and features[1] in ["一般", "固有名詞", "サ変接続", "形容動詞語幹"]) or features[0] == "形容詞":
            if features[6] == "*":
                list_words.append(res.surface)
            else:
                list_words.append(features[6])
        res = res.next
    return list_words

コーパスを読み込んで、形態素解析、bag of wordsを作成

In [47]:
list_corpus = readcsv("./files/file_1.csv")
list_corpuses = [row[1].split("。") for row in list_corpus]
list_corpuses = [sentence for row in list_corpuses for sentence in row]
list_bag_of_words = [parsing(sentence) for sentence in list_corpuses]

bag of wordsから辞書を作成して、高頻度語、低頻度語を除去し、マッピング

In [49]:
dictionary = corpora.Dictionary(list_bag_of_words)
# idで削除
#dictionary.filter_tokens(bad_ids=[4, 95, 93, 64, 32, 112, 178, 36, 28,
#                                  16, 27, 104, 83, 324, 193])
# 頻度で削除
#dictionary.filter_extremes(no_below=3)
dictionary.save_as_text("./files/dictionary1.txt", sort_by_word=False)
dictionary = corpora.Dictionary.load_from_text("./files/dictionary.txt")

辞書からコーパスを作成して、ldaでモデリング、結果をプロット

In [84]:
corpus = [dictionary.doc2bow(text) for text in list_bag_of_words]
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=13)

for i, row in enumerate(lda.show_topics(0)):
    print row